# Import Libraries

In [ ]:
#%%capture
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import os
import tqdm
from BaselineRemoval import BaselineRemoval
import peakutils
from scipy.spatial import ConvexHull

In [ ]:
colors = {
"red" : "#e41a1c",
"blue" : "#377eb8",
"green" : "#4daf4a"
}

fontdict = {"family" : "serif",
           "fontname": "Times New Roman",
          #  "weight" : "bold",
            "color" : "grey",
            "fontsize" : 28
           }

# Functions to extract Data

In [ ]:
def show_data(filePath):
    with open(filePath, 'r') as f:
        content = f.readlines()
        content = np.array([[float(item) for item in c.split()]
                            for c in content if c[0] != '#'])
        return(content)
    
def filter_data(data, low, high):
    temp = data[(data[:, 0] < high) & (data[:, 0] >= low)]
    return(temp)

# Figure-1



# Get Data

In [ ]:
pathPCRnegative = "medFigures/all figures/Figures_for_manuscript_1/PCR Negative 1.txt"
pathPCRnegativeControl ="medFigures/all figures/Figures_for_manuscript_1/Negative Control New.txt"

dataPCRnegative = show_data(pathPCRnegative)
dataPCRnegativeControl = show_data(pathPCRnegativeControl)

print("Initial data shapes\n    Negative: {} \nNegative control: {}"
      .format(dataPCRnegative.shape, dataPCRnegativeControl.shape))

dataPCRnegative = filter_data(dataPCRnegative, 20000, 70000)
dataPCRnegativeControl = filter_data(dataPCRnegativeControl, 20000, 70000)

print("\nData shapes after processing\n    Negative: {} \nNegative control: {}"
      .format(dataPCRnegative.shape, dataPCRnegativeControl.shape))


In [ ]:
plt.figure(figsize=(8.5,5.5), dpi =1000)
plt.rcParams['font.size'] = 12

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
# plt.rcParams['figure.dpi'] = 1000
plt.rcParams["lines.linewidth"] = 0.75
ax = plt.subplot()
ax.plot(np.take(dataPCRnegative,0,axis=1),np.take(dataPCRnegative,1,axis=1), color = colors["red"] )
ax.plot(np.take(dataPCRnegativeControl,0,axis=1),np.take(dataPCRnegativeControl,1,axis=1), color = colors["blue"])
# plt.tick_params(axis = "both")
# from scipy.signal import find_peaks
# px,py = find_peaks(np.take(dataPCRnegative,1,axis=1))
# plt.scatter(px,py)

#peak1
ax.text(0.025,0.0575, "23390", transform=ax.transAxes)
#peak2
ax.text(0.11,0.1, "27917", transform=ax.transAxes)
#peak3
ax.text(0.65,0.97, "55967", transform=ax.transAxes)
#peak4
ax.text(0.715,0.1175, "58270", transform=ax.transAxes)

ax.set_xlabel("m/z",loc = "right")
ax.set_ylabel("ion count",loc = "top", rotation=90)

ax.set_ylim(ymin=0)
ax.set_xlim(xmin=20000)
ax.set_xticks([25000, 35000,45000,55000,65000])
plt.tight_layout()
plt.savefig("medFigures/FINAL/figure1.tiff")
plt.show()

%matplotlib inline

# Figure 2

In [ ]:
r1 = [20000, 150000]
r2 = [33000, 53000]
r3 = [60000, 120000]

peak_vals2 = {"ax0" : {"p1" : 23423,
                      "p2" : 28083,
                      "p3" : 56127,
                      "p4" : 66868,
                       "p5": 79837,
                      "p6" : 113506
                     },
             "ax1" : {"p1" : 33443,
                      "p2" : 40018,
                      "p3" : 47125,
                      "p4" :50929
                     },
            "ax2":  { "p1" : 66868,
                      "p2" : 70603,
                      "p3" : 79837,
                      "p4" : 113506
                     }}
pathPCRneg1 = "medFigures/all figures/Figures_for_manuscript_1/PCR Negative 1.txt"
pathPCRpos3 ="medFigures/all figures/Figures_for_manuscript_1/PCR Positive 3.txt"
pathPCRpos10 ="medFigures/all figures/Figures_for_manuscript_1/PCR Positive 10.txt"

dataPCRneg1 = show_data(pathPCRneg1)
dataPCRpos3 = show_data(pathPCRpos3)
dataPCRpos10 = show_data(pathPCRpos10)

print("Initial data shapes\n    Negative1 : {} \nPositive3: {}\nPositive10: {}"
      .format(dataPCRneg1.shape,dataPCRpos3.shape,dataPCRpos10.shape))

print("Initial data shapes\n    Negative1 : {} \nPositive3: {}\nPositive10: {}"
      .format(dataPCRneg1.shape,dataPCRpos3.shape,dataPCRpos10.shape))

## Helper functions to plot for some figures

In [ ]:
"""
@Arguments : ax :axis , r: range (list), colors [list]
@Returns L axis plotted
"""
def axs_plot_range(dataPCRneg1, dataPCRpos3, dataPCRpos10, ax, r, color,
                   baseline = False, type = "modpoly", correction = False):
    
    dataPCRneg1 = filter_data(dataPCRneg1, r[0], r[1])
    dataPCRpos3 = filter_data(dataPCRpos3, r[0], r[1])
    dataPCRpos10 = filter_data(dataPCRpos10,r[0], r[1])
    
    if baseline:   
        dataPCRpos3 = roll_baseline(dataPCRpos3, type =  type, correction = correction)
        dataPCRpos10 = roll_baseline(dataPCRpos10, type =  type, correction = correction)
    
    print("Type: {}, Correction : {}, Baseline: {}, Minimum value of pos3 {}, min value of pos10{}".format(
        type, str(correction), str(baseline), 
        np.take(dataPCRpos3,1,axis=1).min(), np.take(dataPCRpos10,1,axis=1).min()))
    
    ax.plot(np.take(dataPCRneg1,0,axis=1),np.take(dataPCRneg1,1,axis=1), color = color[0])
    ax.plot(np.take(dataPCRpos3,0,axis=1),np.take(dataPCRpos3,1,axis=1), color = color[1])
    ax.plot(np.take(dataPCRpos10,0,axis=1),np.take(dataPCRpos10,1,axis=1), color = color[2])
    ax.set_xlim(xmin = r[0], xmax = r[1])
    ax.set_ylim(ymin = 0)
    ax.set_xlabel("m/z",loc = "right")
    ax.set_ylabel("ion count",loc = "top", rotation=90)

    return(ax)

## These methods automatically find the baseline, interpolate and adjust the baseline

'''
@ Arguments : arr : array
             type: type of function to use
             correction : To adjust negative values
'''
def roll_baseline(arr, type =  "modpoly", correction = False):
    
    arr = calc_rolling(arr)
    y = np.take(arr, 1, axis = 1)
    x = np.take(arr, 0, axis = 1)
    polynomial_degree=3 #only needed for Modpoly and IModPoly algorithm

    baseObj=BaselineRemoval(y)
    if type=="modpoly":
        out=baseObj.ModPoly(polynomial_degree)
    elif type=="imodpoly":
        out=baseObj.IModPoly(polynomial_degree)
    elif type=="zhang":
        out=baseObj.ZhangFit()
    else:
        out = peakutils.baseline(y)
    
    if correction:
        minval = out.min()
        out = out + abs(minval)
    return(np.hstack((x.reshape(-1,1), out.reshape(-1,1))))


## Plot axis withouth setting x and y limits
def axs_plot_range1(dataPCRneg1, ax, r, color):

    dataPCRneg1 = filter_data(dataPCRneg1, r[0], r[1])
    ax.plot(np.take(dataPCRneg1,0,axis=1),np.take(dataPCRneg1,1,axis=1), color = color[0])
    # ax.plot(np.take(dataPCRpos3,0,axis=1),np.take(dataPCRpos3,1,axis=1), color = color[1])
    #ax.set_xlim(xmin = r[0], xmax = r[1])
    #ax.set_ylim(ymin = 0)
    ax.set_xlabel("m/z",loc = "right")
    ax.set_ylabel("ion count",loc = "top", rotation=90)
    return(ax)

## Smoothening function to calculate moving averages
def calc_rolling(arr):
    print("Check this function : calc_rolling")
    dim1 = np.take(arr,0,axis=1)
    dim2 = pd.Series(np.take(arr,1,axis=1)).rolling(window=150).mean().fillna(0).values # why this? - 750
    return(np.hstack((dim1.reshape(-1,1), dim2.reshape(-1,1))))

# Get limits on x or y axis, for axes plots, 
def get_lims(ax, coord = "x"):
    if coord == "x":
        x = (peak_val - ax.get_xlim()[0])/(ax.get_xlim()[1] - ax.get_xlim()[0])
        return(x)
    if coord == "y":
        return(ax.get_ylim())

## Take the peak value and return the xcoord
def get_text_xcoord(ax, peak_val):
    x = (peak_val - ax.get_xlim()[0])/(ax.get_xlim()[1] - ax.get_xlim()[0])
    x = x - 0.024
    return(x)

## Convoltution for smoothening
def smoothen_signal(signal):
    w = np.ones(len(signal), 'd')
    s = np.convolve(w/w.sum(), signal, mode = 'valid')
    return s


In [ ]:
## Incomplete method/implementation
## Convexhull identifies the border peaks, meaning it will identify both the highs and lows
## Then we have to filter the points occuring at the bottom [incomplete]
## Use interpolation to form a function fitting these points
## Subtract this function from the main function

def smoothen_rubberband(arr):
    dim1 = np.take(arr, 0, axis=1)
    dim2 = np.take(arr, 1, axis=1)
    # Find the convex hull
    v = ConvexHull(np.array(list(zip(dim1, dim2)))).vertices
    
    # Rotate convex hull vertices until they start from the lowest one
    v = np.roll(v, -v.argmin())
    # Leave only the ascending part
   # v = v[:v.argmax()]
    # Convex hull 

    # Create baseline using linear interpolation between vertices
    y = np.interp(dim1, dim1[v], dim2[v])
    
    dim2 = dim2 - y
    print(y.shape)
    return(np.hstack((dim1.reshape(-1,1), dim2.reshape(-1,1)))) 

In [ ]:
c = roll_baseline(filter_data(dataPCRpos3, r2[0],r2[1]),type = "imodpoly", correction=False)
np.take(c, 1, axis =1).min()

In [ ]:
## Sample test case
plt.figure(figsize = (5,5))
ax = plt.subplot( )
#a = filter_data(dataPCRneg1, r2[0],r2[1])

a =  filter_data(dataPCRpos3, r2[0],r2[1])

b =  roll_baseline(filter_data(dataPCRpos3, r2[0],r2[1]))

#c = roll_baseline(filter_data(dataPCRpos10, r2[0],r2[1]))
# ax = axs_plot_range(a,b,c , ax, r2, colorSeq)

ax.plot(np.take(a,0,axis=1),np.take(a,1,axis=1), color = colorSeq[0], label='original')
ax.plot(np.take(b,0,axis=1),np.take(b,1,axis=1), color = colorSeq[0], label='corr')
#ax.plot(np.take(c,0,axis=1),np.take(c,1,axis=1), color = colorSeq[0])
# ax.set_xlim(xmin = r[0], xmax = r[1])
# ax.set_ylim(ymin = 0)
ax.set_xlabel("m/z",loc = "right")
ax.set_ylabel("a.i.",loc = "top", rotation=90)
plt.legend()
plt.show()

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
colorSeq

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 1000
plt.rcParams["lines.linewidth"] = 0.4
_, axs = plt.subplots(3,1, figsize= (8,10), dpi =1000)


axs[0].set_ylim(ymin=0, ymax=340000)
axs[1].set_ylim(ymin=0, ymax=3500)
axs[2].set_ylim(ymin=0, ymax=15000)

colorSeq = [colors["blue"], colors["red"], colors["green"]]
axs[0] = axs_plot_range(dataPCRneg1, dataPCRpos3, dataPCRpos10, axs[0], r1, colorSeq )
# dataPCRpos3Filtered = dataPCRpos3
axs[1] = axs_plot_range(dataPCRneg1, dataPCRpos3, dataPCRpos10, 
                        axs[1], r2, colorSeq, 
                        baseline=True, type = "modpoly", correction=True)

axs[2] = axs_plot_range(dataPCRneg1, dataPCRpos3, dataPCRpos10,axs[2], r3, colorSeq,
                       baseline=True, type = "modpoly", correction=True)

axs[0].set_xticks([25000, 45000, 65000, 85000, 105000, 125000, 145000])
axs[2].set_xticks([65000, 75000, 85000, 95000, 105000, 115000])

#ax0
#peak1
axs[0].text(get_text_xcoord(axs[0], peak_vals2["ax0"]["p1"]),0.2, "23423", transform=axs[0].transAxes)
#peak2
axs[0].text(get_text_xcoord(axs[0], peak_vals2["ax0"]["p2"]),0.125, "28083", transform=axs[0].transAxes)
#peak3
axs[0].text(get_text_xcoord(axs[0], peak_vals2["ax0"]["p3"]),0.92, "56127", transform=axs[0].transAxes)
#peak4
axs[0].text(get_text_xcoord(axs[0], peak_vals2["ax0"]["p4"]),0.06, "66868", transform=axs[0].transAxes)
#peak5
axs[0].text(get_text_xcoord(axs[0], peak_vals2["ax0"]["p5"]),0.03, "79837", transform=axs[0].transAxes)
#peak6
axs[0].text(get_text_xcoord(axs[0], peak_vals2["ax0"]["p6"]),0.055, "113506", transform=axs[0].transAxes)

#ax1
#peak1
axs[1].text(get_text_xcoord(axs[1], 
                            peak_vals2["ax1"]["p1"]),0.84, "33443", transform=axs[1].transAxes)
#peak2
axs[1].text(get_text_xcoord(axs[1], peak_vals2["ax1"]["p2"]),0.7, "40018", transform=axs[1].transAxes)
#peak3
axs[1].text(get_text_xcoord(axs[1], peak_vals2["ax1"]["p3"]),0.455, "47125", transform=axs[1].transAxes)
#peak4
axs[1].text(get_text_xcoord(axs[1], peak_vals2["ax1"]["p4"]),0.455, "50929", transform=axs[1].transAxes)

#ax2
#peak1
axs[2].text(get_text_xcoord(axs[2], peak_vals2["ax2"]["p1"]),0.78, 
            str(peak_vals2["ax2"]["p1"]), transform=axs[2].transAxes)
#peak2
axs[2].text(get_text_xcoord(axs[2], peak_vals2["ax2"]["p2"]),0.15, 
                            str(peak_vals2["ax2"]["p2"]), transform=axs[2].transAxes)
#peak3
axs[2].text(get_text_xcoord(axs[2], peak_vals2["ax2"]["p3"]),0.2,
            str(peak_vals2["ax2"]["p3"]), transform=axs[2].transAxes)
#peak4
axs[2].text(get_text_xcoord(axs[2], peak_vals2["ax2"]["p4"]),0.715,
            str(peak_vals2["ax2"]["p4"]), transform=axs[2].transAxes)

fontdict = {"family" : "serif",
           "fontname": "Times New Roman",
          #  "weight" : "bold",
            "color" : "grey",
            "fontsize" : 28
           }

axs[0].text(0.925, 0.85, "A", transform=axs[0].transAxes, fontdict=fontdict)

axs[1].text(0.925, 0.85, "B", transform=axs[1].transAxes, fontdict=fontdict)

axs[2].text(0.925, 0.85, "C", transform=axs[2].transAxes, fontdict=fontdict)

plt.tight_layout()
plt.savefig("medFigures/FINAL/figure2.tiff")
# df.ewm(span=15, adjust = False).mean()

plt.show()
%matplotlib inline

# Figure 3

In [ ]:
r1 = [20000, 160000]
r2 = [20000, 75000]
r3 = [60000, 155000]

pathHumIgM = "medFigures/all figures/Figures_for_manuscript_1/Human serum IgM.txt"
pathHumIgA ="medFigures/all figures/Figures_for_manuscript_1/Human serum IgA.txt"
pathHumIgG ="medFigures/all figures/Figures_for_manuscript_1/Huamn serum IgG.txt"

dataHumIgA = show_data(pathHumIgA) #blue
dataHumIgM = show_data(pathHumIgM) #red
dataHumIgG = show_data(pathHumIgG) #green

In [ ]:
peak_vals3 = {"ax0" : {"p1" : 23343,
                      "p2" : 46758,
                      "p3" : 51142,
                      "p4" : 57372,
                      "p5" : 71678,
                       "p6" : 80967,
                       "p7" : 102700,
                       "p8": 115158,
                       "p9" : 126358
                     },
             "ax1" : {"p1" : 23343,
                      "p2" : 46758,
                      "p3" : 51152,
                      "p4" :57372
                     },
            "ax2":  { "p1" : 67134,
                      "p2" : 71678,
                      "p3" : 74827,
                      "p4" : 80967,
                     "p5" : 94807,
                     "p6" : 102700,
                     "p7" : 115158,
                     "p8" : 126358, 
                     "p9" : 138579
                     }}

In [ ]:
plt.rcParams['font.size'] = 12

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 1000
plt.rcParams["lines.linewidth"] = 0.5

fig3 = plt.figure(constrained_layout= False)
fig3.set_size_inches(8,9)
gs = fig3.add_gridspec(2,2)
ax0 = fig3.add_subplot(gs[0,:]) 
ax1 = fig3.add_subplot(gs[1,0])
ax2 = fig3.add_subplot(gs[1,1])

# _, axs = plt.subplots(3,1, figsize= (8,10), dpi =300)


# axs[0].set_ylim(ymin=0, ymax=340000)
# axs[1].set_ylim(ymin=0, ymax=5500)
# axs[2].set_ylim(ymin=0, ymax=17000)
colorSeq = colorSeq = [colors["blue"], colors["red"], colors["green"]]
ax0 = axs_plot_range(dataHumIgA, dataHumIgM, dataHumIgG, ax0, r1, colorSeq)
ax1 = axs_plot_range(dataHumIgA, dataHumIgM, dataHumIgG, ax1, r2, colorSeq)

ax2 = axs_plot_range(dataHumIgA, dataHumIgM, dataHumIgG, ax2, r3, colorSeq)

# axs[0] = axs_plot_range(dataPCRneg1, dataPCRpos3, dataPCRpos10, axs[0], r1, colors)
# axs[1] = axs_plot_range(dataPCRneg1, dataPCRpos3, dataPCRpos10,axs[1], r2, colors)
# axs[2] = axs_plot_range(dataPCRneg1, dataPCRpos3, dataPCRpos10,axs[2], r3, colors)

# axs[0].set_xticks([25000, 45000, 65000, 85000, 105000, 125000, 145000])
# axs[2].set_xticks([65000, 75000, 85000, 95000, 105000, 115000])
rot = 90
# #ax0
# #peak1
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p1"]), 0.95, 
        str(peak_vals3["ax0"]["p1"]), transform=ax0.transAxes)
# #peak2
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p2"]), 0.1, 
        str(peak_vals3["ax0"]["p2"]), transform=ax0.transAxes, rotation = rot)# #peak3
#peak3
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p3"]), 0.675, 
        str(peak_vals3["ax0"]["p3"]), transform=ax0.transAxes)# #peak3

#peak4
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p4"]), 0.475, 
        str(peak_vals3["ax0"]["p4"]), transform=ax0.transAxes)# #peak3

#peak5
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p5"]), 0.15, 
        str(peak_vals3["ax0"]["p5"]), transform=ax0.transAxes)# #peak3

#peak6
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p6"]), 0.1, 
        str(peak_vals3["ax0"]["p6"]), transform=ax0.transAxes)# #peak3

#peak7
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p7"]), 0.115, 
        str(peak_vals3["ax0"]["p7"]), transform=ax0.transAxes)# #peak3

#peak8
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p8"]), 0.065, 
        str(peak_vals3["ax0"]["p8"]), transform=ax0.transAxes)# #peak3

#peak9
ax0.text(get_text_xcoord(ax0, peak_vals3["ax0"]["p9"]), 0.015, 
        str(peak_vals3["ax0"]["p9"]), transform=ax0.transAxes)# #peak3


# #ax1
# #peak1
ax1.text(get_text_xcoord(ax1, peak_vals3["ax1"]["p1"]), 0.95, 
        str(peak_vals3["ax1"]["p1"]), transform=ax1.transAxes)
# #peak2
ax1.text(get_text_xcoord(ax1, peak_vals3["ax1"]["p2"]), 0.1, 
        str(peak_vals3["ax1"]["p2"]), rotation = 90,transform=ax1.transAxes)# #peak3
#peak3
ax1.text(get_text_xcoord(ax1, peak_vals3["ax1"]["p3"]), 0.675, 
        str(peak_vals3["ax1"]["p3"]), transform=ax1.transAxes)# #peak3

#peak4
ax1.text(get_text_xcoord(ax1, peak_vals3["ax1"]["p4"]), 0.475, 
        str(peak_vals3["ax1"]["p4"]), transform=ax1.transAxes)# #peak3



# #ax1
# #peak1
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p1"])-0.0125, 0.15, 
        str(peak_vals3["ax2"]["p1"]), rotation = 90,transform=ax2.transAxes)
# #peak2
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p2"]), 0.95, 
        str(peak_vals3["ax2"]["p2"]), transform=ax2.transAxes)# #peak3
#peak3
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p3"]), 0.875, 
        str(peak_vals3["ax2"]["p3"]), transform=ax2.transAxes)# #peak3

#peak4
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p4"]), 0.675, 
        str(peak_vals3["ax2"]["p4"]), transform=ax2.transAxes)# #peak3

#peak5
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p5"]), 0.125, 
        str(peak_vals3["ax2"]["p5"]), rotation = rot, transform=ax2.transAxes)# #peak3
#peak6
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p6"]), 0.475, 
        str(peak_vals3["ax2"]["p6"]), transform=ax2.transAxes)# #peak3
#peak7
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p7"]), 0.225, 
        str(peak_vals3["ax2"]["p7"]), transform=ax2.transAxes)# #peak3
#peak8
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p8"]), 0.1075, 
        str(peak_vals3["ax2"]["p8"]), transform=ax2.transAxes)# #peak3
#peak
ax2.text(get_text_xcoord(ax2, peak_vals3["ax2"]["p9"]) - 0.05, 0.0525, 
        str(peak_vals3["ax2"]["p9"]), transform=ax2.transAxes)# #peak3

ax0.text(0.925, 0.85, "A", transform=ax0.transAxes, fontdict=fontdict)
ax1.text(0.865, 0.85, "B", transform=ax1.transAxes, fontdict=fontdict)
ax2.text(0.865, 0.85, "C", transform=ax2.transAxes, fontdict=fontdict)

ax0.set_xticks([25000, 45000, 65000, 85000, 105000, 125000, 145000])
ax1.set_xticks([25000, 40000, 55000, 70000])
ax2.set_xticks([65000, 90000, 115000, 140000])

plt.tight_layout()
fig3.savefig("medFigures/FINAL/figure3.tiff")
plt.show()
%matplotlib inline

# Figure 4

In [ ]:
r1 = [20000, 160000]
r2 = [20000, 75000]
r3 = [60000, 155000]

pathPosControl = "medFigures/all figures/Figures_for_manuscript_1/Positive Control.txt"

dataPosControl = show_data(pathPosControl) #blue

In [ ]:
r1 = [20000, 150000]
r2 = [70000, 150000]

peak_vals4 = {"ax0" : {"p1" : 33425,
                      "p2" : 45754,
                      "p3" : 53840,
                      "p4" : 66574,
                      "p5" : 78531,
                       "p6" : 100219
                     },
             "ax1" : {"p1" : 78531,
                      "p2" : 132950
                     }}

In [ ]:
plt.rcParams['font.size'] = 12

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 600
plt.rcParams["lines.linewidth"] = 0.4
_, axs = plt.subplots(2,1, figsize= (8,7.5), dpi =1000)


# axs[0].set_ylim(ymin=0, ymax=340000)
axs[0].set_ylim(ymin=0, ymax=40000)
axs[1].set_ylim(ymin=0, ymax=550)

# axs[2].set_ylim(ymin=0, ymax=17000)

colorSeq = [colors["blue"],colors["blue"]]

ax0Data = filter_data(dataPosControl, r1[0], r1[1])
axs[0].plot(np.take(ax0Data,0,axis=1),np.take(ax0Data,1,axis=1),linewidth=0.8,
            color = colorSeq[0])
axs[0].set_xlabel("m/z",loc = "right")
axs[0].set_ylabel("ion count",loc = "top", rotation=90)
axs[0].set_xlim(xmin = r1[0], xmax = r1[1])



ax1Data = filter_data(dataPosControl, r2[0], r2[1])
axs[1].plot(np.take(ax1Data,0,axis=1),np.take(ax1Data,1,axis=1),linewidth=0.4,
            color = colorSeq[0])
axs[1].set_xlabel("m/z",loc = "right")
axs[1].set_ylabel("ion count",loc = "top", rotation=90)
axs[1].set_xlim(xmin = r2[0], xmax = r2[1])



axs[0].set_xticks([25000, 45000, 65000, 85000, 105000, 125000, 145000])
axs[1].set_xticks([75000, 85000, 95000, 105000, 115000, 125000, 135000,145000])

#ax1
#peak1
axs[0].text(get_text_xcoord(axs[0], 
                            peak_vals4["ax0"]["p1"]),0.125, 
            str(peak_vals4["ax0"]["p1"]), transform=axs[0].transAxes)

#peak2
axs[0].text(get_text_xcoord(axs[0], 
                            peak_vals4["ax0"]["p2"]),0.125, 
            str(peak_vals4["ax0"]["p2"]), transform=axs[0].transAxes)
#peak3
axs[0].text(get_text_xcoord(axs[0], 
                            peak_vals4["ax0"]["p3"]),0.055, 
            str(peak_vals4["ax0"]["p3"]), transform=axs[0].transAxes)
#peak4
axs[0].text(get_text_xcoord(axs[0], 
                            peak_vals4["ax0"]["p4"]),0.90, 
            str(peak_vals4["ax0"]["p4"]), transform=axs[0].transAxes)
#peak5
axs[0].text(get_text_xcoord(axs[0], 
                            peak_vals4["ax0"]["p5"]),0.05, 
            str(peak_vals4["ax0"]["p5"]), transform=axs[0].transAxes)
# #peak6
# axs[0].text(get_text_xcoord(axs[0], 
#                             peak_vals2["ax0"]["p6"]),0.025, 
#             str(peak_vals2["ax0"]["p6"]), transform=axs[0].transAxes)




#ax1
#peak1
axs[1].text(get_text_xcoord(axs[1], 
                            peak_vals4["ax1"]["p1"]),0.865,
            str(peak_vals4["ax1"]["p1"]), transform=axs[1].transAxes)
#peak2
axs[1].text(get_text_xcoord(axs[1], peak_vals4["ax1"]["p2"]),0.15, 
            str(peak_vals4["ax1"]["p2"]), transform=axs[1].transAxes)


axs[0].text(0.925, 0.85, "A", transform=axs[0].transAxes, fontdict=fontdict)

axs[1].text(0.925, 0.85, "B", transform=axs[1].transAxes, fontdict=fontdict)


plt.tight_layout()
plt.savefig("medFigures/FINAL/Figure4.tiff")

plt.show()
%matplotlib inline

# Figure 5

In [ ]:
PATH = "AllResults/Files/callibrant/data/main/files60AUCRed.csv"
from magic import magicData
data = pd.read_csv(PATH)
data.head(3)
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams["scatter.edgecolors"] = 'black'
plt.rcParams["lines.linewidth"] = 10
plt.rcParams["figure.dpi"] = 1000

feature_names = data.columns[1:8]
colorsL = {
"red" : "#e41a1c",
"blue" : "#377eb8",
"green" : "#4daf4a"
}
colors = [colorsL["blue"] if f=='PCR_Negative' else colorsL["red"] for f in data['label']]

legend = [Line2D([1],[1], marker='o',  markerfacecolor=colorsL["blue"], markersize=8, color = 'w', label = 'COVID-19 Negative'),
         Line2D([1],[2],  marker='o',  markerfacecolor = colorsL["red"],markersize=8, color = 'w',label= 'COVID-19 Positive')]

fig = plt.figure(figsize=(8,8), dpi = 1000)
iplot = 320
plt.subplots_adjust(wspace=0.2,hspace=0.4)
#['2', '3', '4', '5', '6']
for i,fn in enumerate([1,2,3,4,5,6]):
    iplot += 1
    
    if i == 5:
        ax = fig.add_subplot(iplot)
        ax.plot()#plt.subplot2grid((3,6), (i//2, 1), colspan=3)
        ax.legend(handles = legend, loc = 'center', ncol = 1,framealpha=0.0,frameon=True,fancybox=True)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)    
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)  
        ax.set_xticks([])
        ax.set_yticks([])

    else:
    
        # You can be fancy and use subplot2grid for each plot, which doesn't
        # require keeping the iplot variable:
        # ax = plt.subplot2grid((4,2), (i//2,i%2))

        # Or you can keep using add_subplot, which may be simpler:
        ax = fig.add_subplot(iplot)
        ax.set_title("{} m/z - {} m/z".format(magicData.important_ranges_new[int(i)][0],magicData.important_ranges_new[int(i)][1]))
        ax.scatter(data.index,data.iloc[:,fn], color=colors, s=16)
    
    
#plt.tight_layout()
plt.savefig("extras/Figure5mod.tiff")

In [ ]:
data.head()

In [ ]:
data.iloc[:,1]

# Figure 6

In [ ]:

#pathAmylase = "medFigures/all figures/Figures_for_manuscript_1/Amylase100pmol.txt"
pathAmylase = "extras/S37_baseline corrected_low viral.txt"


dataAmylase = show_data(pathAmylase) #blue

r1 = [75000, 90000]

# peak_vals5 = {"ax0" : {"p1" : 28314,
#                       "p2" : 56625
#                        #,
#                       #"p3" : 67422,
#                       #"p4" : 70808
#                      }}



In [ ]:
colors = {
"red" : "#e41a1c",
"blue" : "#377eb8",
"green" : "#4daf4a"
}

In [ ]:
plt.rcParams['font.size'] = 8

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 1000
plt.rcParams["lines.linewidth"] = 0.4
_, axs = plt.subplots(1,1, figsize= (4,2.5), dpi =1000)

# axs.set_ylim(ymin=0, ymax=320000)
axs.set_ylim(ymin=0, ymax=1300)

colorSeq = [colors["blue"]]

ax0Data = filter_data(dataAmylase, r1[0], r1[1])

axs.plot(np.take(ax0Data,0,axis=1),np.take(ax0Data,1,axis=1),linewidth=0.8,
            color = colorSeq[0])
axs.set_xlabel("m/z",loc = "right")
axs.set_ylabel("ion count",loc = "top", rotation=90)
axs.set_xlim(xmin = r1[0], xmax = r1[1])


axs.set_xticks([70000, 80000,90000])

#axs.set_xticks([25000, 35000, 45000, 55000, 65000, 75000, 85000,95000,105000,115000])

# #ax1
# #peak1
# axs.text(get_text_xcoord(axs, 
#                             peak_vals5["ax0"]["p1"]),0.12, 
#             str(peak_vals5["ax0"]["p1"]), transform=axs.transAxes)

# #peak2
# axs.text(get_text_xcoord(axs, 
#                             peak_vals5["ax0"]["p2"]),0.835, 
#             str(peak_vals5["ax0"]["p2"]), transform=axs.transAxes)

# #ax0
# #peak1
# axs.text(get_text_xcoord(axs, 
#                             peak_vals5["ax0"]["p1"]),0.13, 
#             str(peak_vals5["ax0"]["p1"]), transform=axs.transAxes)

# #peak2
# axs.text(get_text_xcoord(axs, 
#                             peak_vals5["ax0"]["p2"]),0.945, 
#             str(peak_vals5["ax0"]["p2"]), transform=axs.transAxes)

#peak3
axs.text(get_text_xcoord(axs, 
                            peak_vals5["ax0"]["p3"]),0.275,
            str(peak_vals5["ax0"]["p3"]), transform=axs.transAxes)
#peak4
axs.text(get_text_xcoord(axs, 
                            peak_vals5["ax0"]["p4"]),0.95, 
            str(peak_vals5["ax0"]["p4"]), transform=axs.transAxes)
#peak5
axs.text(get_text_xcoord(axs, 
                            peak_vals5["ax0"]["p5"]),0.15, 
            str(peak_vals5["ax0"]["p5"]), transform=axs.transAxes)
#peak6
axs.text(get_text_xcoord(axs, 
                            peak_vals5["ax0"]["p6"]),0.3, 
            str(peak_vals5["ax0"]["p6"]), transform=axs.transAxes)
# #peak7
# axs.text(get_text_xcoord(axs, 
#                             peak_vals5["ax0"]["p7"]),0.05, 
#             str(peak_vals5["ax0"]["p7"]), transform=axs.transAxes)



# axs[0].text(0.925, 0.85, "A", transform=axs[0].transAxes, fontdict=fontdict)

# axs[1].text(0.925, 0.85, "B", transform=axs[1].transAxes, fontdict=fontdict)


plt.tight_layout()
plt.savefig("extras/subFigmin.tiff")

plt.show()
%matplotlib inline

# Supplementary figure - 1

In [ ]:

#pathAmylase = "medFigures/all figures/Figures_for_manuscript_1/Amylase100pmol.txt"
pathAmylase = "extras/S37_baseline corrected_low viral.txt"


dataAmylase = show_data(pathAmylase) #blue

r1 = [20000, 120000]

r2 = [75000, 90000]

peak_vals5 = {"ax0" : {"p1" : 27998,
                       "p2" : 56081,
                       "p3" : 67117,
                       "p4" : 70432,
                       "p5" : 79660,
                       "p6" : 85042,
                       "p7" : 112707 
                      }}
colors = {
"red" : "#e41a1c",
"blue" : "#377eb8",
"green" : "#4daf4a"
}

In [ ]:
plt.rcParams['font.size'] = 12

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 1000
plt.rcParams["lines.linewidth"] = 0.4


plt.figure(figsize = (8,3.5))

#_, axs = plt.subplots(1,1, figsize= (8,3.5), dpi =1000)

#axs.set_ylim(ymin=0, ymax=1300)
diff1 = 0.024* (r1[1]-r1[0])
diff2 = 0.024* (r2[1]- r2[0])
colorSeq = [colors["green"]]

ax0Data = filter_data(dataAmylase, r1[0], r1[1])
ax1Data = filter_data(dataAmylase, r2[0], r2[1])


plt.plot(np.take(ax0Data,0,axis=1),np.take(ax0Data,1,axis=1),linewidth=0.8,
            color = colorSeq[0])

plt.xlabel("m/z",loc = "right")
plt.ylabel("ion count",loc = "top", rotation=90)
plt.xlim(xmin = r1[0], xmax = r1[1])
plt.ylim(ymin=0, ymax=320000)

plt.xticks([20000, 40000,  60000, 80000,100000,120000])

# #ax1
# #peak1
plt.text( peak_vals5["ax0"]["p1"]-diff1,50000, 
            str(peak_vals5["ax0"]["p1"]))

#peak2
plt.text(peak_vals5["ax0"]["p2"]-diff1,300500, str(peak_vals5["ax0"]["p2"]))


#peak3
plt.text(peak_vals5["ax0"]["p3"]-diff2,4000, str(peak_vals5["ax0"]["p3"]))

# #peak4
# plt.text(peak_vals5["ax0"]["p4"]-diff2,20000, str(peak_vals5["ax0"]["p4"]))



#peak7
plt.text(peak_vals5["ax0"]["p7"]-diff1,15000, str(peak_vals5["ax0"]["p7"]))


##########################
plt.rcParams['figure.dpi'] = 1000
plt.rcParams['font.size'] = 8
a = plt.axes([.475, .45, .45, .45])
f = plt.plot(np.take(ax1Data,0,axis=1),np.take(ax1Data,1,axis=1),linewidth=0.8,
            color = colorSeq[0])

plt.yticks([0,200,400],fontsize=6)
plt.xlim(xmin = r2[0], xmax = r2[1])
plt.ylim(ymin=0, ymax=500)
plt.xticks([75000, 80000, 85000,90000],fontsize=6
          )
#plt.xticks([])
#plt.yticks([])

# #peak3
# plt.text(peak_vals5["ax0"]["p3"]-diff2,400, str(peak_vals5["ax0"]["p3"]))

# #peak4
# plt.text(peak_vals5["ax0"]["p4"]-diff2,1250, str(peak_vals5["ax0"]["p4"]))

# #peak5
plt.text(peak_vals5["ax0"]["p5"]-diff2,180, str(peak_vals5["ax0"]["p5"]))
# #peak6
plt.text(peak_vals5["ax0"]["p6"]-diff2,400, str(peak_vals5["ax0"]["p6"]))


#plt.xlabel("m/z",loc = "right")
#plt.ylabel("ion count",loc = "top", rotation=90)

# axs[0].text(0.925, 0.85, "A", transform=axs[0].transAxes, fontdict=fontdict)

# axs[1].text(0.925, 0.85, "B", transform=axs[1].transAxes, fontdict=fontdict)


plt.tight_layout()
plt.savefig("extras/supV6.tiff")

plt.show()
%matplotlib inline